# Overview



# Input

In [80]:
pdb_name = "6scm" # f'{pdb_name}.pdb' file in the inputs folder will be used.
target_name = "SOS1"  # "target" in the .csv file
csv_name = "all_middle.test.csv"

given_pocket = 0
#given_pocket = "pocket_8" # or int 0

input_path = "./inputs" # for .pdb files 
base_pre = f"./AFFINITY" # for results

conf_by_chosen = True
# if false and "given_pocket" is 0, only the conformations relating to the given_pocket will be generated.

distinguish_by_timestamp = True 
# if true, the output folder name will be suffiexed with a timestamp 


p2rank = "bash ./p2rank_2.3/prank" # p2rank file path

In [81]:
path_label_data = "../../sar-3d-pharam/data/label_data/"
path_data_folder = "SOS1 HPK1" # or "" if the .csv file is placed in label_data folder


    # label_data*
    # - HPK1 SOS1*
    #     - all_middle.test.csv
    #     - all_middle.train.csv
    #     - ...
    # - PRTM5*
    #     - all_middle.test.csv
    #     - all_middle.train.csv
    #     - ...
    # - all_middle.test.csv
    # - all_middle.train.csv
    # - ...

path_data = path_label_data + path_data_folder
import os
os.listdir(path_data)
    # You should see something like this:
    #   ['all_middle.test.csv',
    #    'all_middle.train.csv',
    #    ...
    # The SMILES in the 'all_middle.text.csv' file will be used ：

['.ipynb_checkpoints',
 'all_middle.test.csv',
 'all_middle.train.csv',
 'all_ndcg.test.csv',
 'all_ndcg.train.csv',
 'all_pair.test.csv',
 'all_pair.train.csv',
 'all_top.test.csv',
 'all_top.train.csv']

In [82]:
cp_name = target_name.replace("_","")
tankbind_src_folder_path = "../tankbind/"
import sys
sys.path.insert(0, tankbind_src_folder_path)
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import time

In [83]:
cp_name

'SOS1'

In [84]:

timetag = time.strftime("%m%d%H%M")

if distinguish_by_timestamp:
    pre = f"{base_pre}/{cp_name}-{pdb_name}-{timetag}"
else:
    pre = f"{base_pre}/{cp_name}-{pdb_name}"
    
os.system(f"mkdir -p {pre}")
os.system(f"rm -rf {pre}/sdfs")
os.system(f"mkdir -p {pre}/sdfs")
os.system(f"rm -rf {pre}/PDBs")
os.system(f"mkdir -p {pre}/PDBs")
os.system(f"rm -rf {pre}/p2rank")
os.system(f"mkdir -p {pre}/p2rank")

proteinName = pdb_name
proteinFile = f"{pre}/{proteinName}.pdb"

# get protein feature

In [85]:
from feature_utils import get_protein_feature

In [86]:
from Bio.PDB import PDBParser
from feature_utils import get_clean_res_list
parser = PDBParser(QUIET=True)
protein_dict = {}
proteinName = pdb_name
proteinFile = f"./inputs/{pdb_name}.pdb"
s = parser.get_structure("example", proteinFile)
res_list = list(s.get_residues())
clean_res_list = get_clean_res_list(res_list, ensure_ca_exist=True)
protein_dict[proteinName] = get_protein_feature(clean_res_list)

# p2rank to segment the protein

In [87]:
ds = f"{pre}/protein_list.ds"
with open(ds, "w") as out:
    out.write(f"../../{input_path}/{pdb_name}.pdb\n")

In [88]:
cmd = f"{p2rank} predict {ds} -o {pre}/p2rank -threads 1"
os.system(cmd)

----------------------------------------------------------------------------------------------
 P2Rank 2.3
----------------------------------------------------------------------------------------------

predicting pockets for proteins from dataset [protein_list.ds]
processing [6scm.pdb] (1/1)
predicting pockets finished in 0 hours 0 minutes 11.957 seconds
results saved to directory [/home/jovyan/TankBind/examples/AFFINITY/SOS1-6scm-07260945/p2rank]

----------------------------------------------------------------------------------------------
 finished successfully in 0 hours 0 minutes 12.817 seconds
----------------------------------------------------------------------------------------------


0

In [89]:
d = pd.read_csv(f'{path_data}/{csv_name}')
d = d[d["target"]==target_name]

In [90]:
info = []
for i, line in tqdm(d.iterrows(), total=d.shape[0]):
    smiles = line['smiles']
    compound_name = ""
    protein_name = proteinName
    # use protein center as the pocket center.
#    com = ",".join([str(a.round(3)) for a in protein_dict[proteinName][0].mean(axis=0).numpy()])
 #   info.append([protein_name, compound_name, smiles, "protein_center", com])
    # since WDR is actually small enough, and we are interested in finding a ligand binds to the central cavity.
    # the block centered at the centroid of the protein is enough.
    # we don't need additional p2rank predicted centers.
    if True:
        p2rankFile = f"{pre}/p2rank/{proteinName}.pdb_predictions.csv"
        pocket = pd.read_csv(p2rankFile)
        pocket.columns = pocket.columns.str.strip()
        pocket_coms = pocket[['center_x', 'center_y', 'center_z']].values
        for ith_pocket, com in enumerate(pocket_coms):
            com = ",".join([str(a.round(3)) for a in com])
            info.append([protein_name, compound_name, smiles, f"pocket_{ith_pocket+1}", com])
info = pd.DataFrame(info, columns=['protein_name', 'compound_name', 'smiles', 'pocket_name', 'pocket_com'])
info

100%|██████████| 235/235 [00:00<00:00, 302.47it/s]


,protein_name,compound_name,smiles,pocket_name,pocket_com
0,6scm,,CN1CCN(c2ccc(Nc3ncc(-c4ccnc5[nH]ccc45)c4cc[nH]...,pocket_1,"8.709,-24.923,-34.936"
1,6scm,,CN1CCN(c2ccc(Nc3ncc(-c4ccnc5[nH]ccc45)c4cc[nH]...,pocket_2,"14.982,-43.708,-30.367"
2,6scm,,CN1CCN(c2ccc(Nc3ncc(-c4ccnc5[nH]ccc45)c4cc[nH]...,pocket_3,"4.658,-7.672,-15.484"
3,6scm,,CN1CCN(c2ccc(Nc3ncc(-c4ccnc5[nH]ccc45)c4cc[nH]...,pocket_4,"1.534,-35.549,-43.763"
4,6scm,,CN1CCN(c2ccc(Nc3ncc(-c4ccnc5[nH]ccc45)c4cc[nH]...,pocket_5,"7.982,19.069,-15.421"
...,...,...,...,...,...
2110,6scm,,C[C@@H](NC(=O)c1cc(N2CCC(C)(O)C2)nc2cc[nH]c12)...,pocket_5,"7.982,19.069,-15.421"
2111,6scm,,C[C@@H](NC(=O)c1cc(N2CCC(C)(O)C2)nc2cc[nH]c12)...,pocket_6,"-1.262,-16.002,-29.16"
2112,6scm,,C[C@@H](NC(=O)c1cc(N2CCC(C)(O)C2)nc2cc[nH]c12)...,pocket_7,"3.006,3.753,-22.598"
2113,6scm,,C[C@@H](NC(=O)c1cc(N2CCC(C)(O)C2)nc2cc[nH]c12)...,pocket_8,"15.256,-6.178,-31.047"


# construct dataset

In [91]:
import torch
torch.set_num_threads(1)

In [92]:
from torch_geometric.data import Dataset
from utils import construct_data_from_graph_gvp
import rdkit.Chem as Chem    # conda install rdkit -c rdkit if import failure.
from feature_utils import extract_torchdrug_feature_from_mol, get_canonical_smiles

In [93]:
savelist = []

In [94]:
class MyDataset_VS(Dataset):
    def __init__(self, root, data=None, protein_dict=None, proteinMode=0, compoundMode=1,
                pocket_radius=20, shake_nodes=None,
                 transform=None, pre_transform=None, pre_filter=None):
        self.data = data
        self.protein_dict = protein_dict
        super().__init__(root, transform, pre_transform, pre_filter)
        print(self.processed_paths)
        self.data = torch.load(self.processed_paths[0])
        self.protein_dict = torch.load(self.processed_paths[1])
        self.proteinMode = proteinMode
        self.pocket_radius = pocket_radius
        self.compoundMode = compoundMode
        self.shake_nodes = shake_nodes
        #self.printflag = True
    @property
    def processed_file_names(self):
        return ['data.pt', 'protein.pt']

    def process(self):
        torch.save(self.data, self.processed_paths[0])
        torch.save(self.protein_dict, self.processed_paths[1])

    def len(self):
        return len(self.data)

    def get(self, idx):
        line = self.data.iloc[idx]
        smiles = line['smiles']
        pocket_com = line['pocket_com']
        pocket_com = np.array(pocket_com.split(",")).astype(float) if type(pocket_com) == str else pocket_com
        pocket_com = pocket_com.reshape((1, 3))
        use_whole_protein = line['use_whole_protein'] if "use_whole_protein" in line.index else False

        protein_name = line['protein_name']
        protein_node_xyz, protein_seq, protein_node_s, protein_node_v, protein_edge_index, protein_edge_s, protein_edge_v = self.protein_dict[protein_name]

        
        
        try:
            smiles = get_canonical_smiles(smiles)
            mol = Chem.MolFromSmiles(smiles)
            mol.Compute2DCoords()
                
            coords, compound_node_features, input_atom_edge_list, input_atom_edge_attr_list, pair_dis_distribution = extract_torchdrug_feature_from_mol(mol, has_LAS_mask=True)
        except:
            print("something wrong with ", smiles, "to prevent this stops our screening, we repalce it with a placeholder smiles 'CCC'")
            smiles = 'CCC'
            mol = Chem.MolFromSmiles(smiles)
            mol.Compute2DCoords()
            coords, compound_node_features, input_atom_edge_list, input_atom_edge_attr_list, pair_dis_distribution = extract_torchdrug_feature_from_mol(mol, has_LAS_mask=True)
        # y is distance map, instead of contact map.
        data, input_node_list, keepNode = construct_data_from_graph_gvp(protein_node_xyz, protein_seq, protein_node_s, 
                              protein_node_v, protein_edge_index, protein_edge_s, protein_edge_v,
                              coords, compound_node_features, input_atom_edge_list, input_atom_edge_attr_list,
                              pocket_radius=self.pocket_radius, use_whole_protein=use_whole_protein, includeDisMap=True,
                              use_compound_com_as_pocket=False, chosen_pocket_com=pocket_com, compoundMode=self.compoundMode)
        data.compound_pair = pair_dis_distribution.reshape(-1, 16)
        return data


In [95]:
dataset_path = f"{pre}/dataset/"
os.system(f"rm -r {dataset_path}")
os.system(f"mkdir -p {dataset_path}")
dataset = MyDataset_VS(dataset_path, data=info, protein_dict=protein_dict)

rm: cannot remove './AFFINITY/SOS1-6scm-07260945/dataset/': No such file or directory
Processing...
Done!


['AFFINITY/SOS1-6scm-07260945/dataset/processed/data.pt', 'AFFINITY/SOS1-6scm-07260945/dataset/processed/protein.pt']


In [96]:
import logging
from torch_geometric.loader import DataLoader
from tqdm import tqdm    # pip install tqdm if fails.
from model import get_model

# take about 2 minutes to screen 10,000 drug candidates.

In [97]:
bias_list = []
batched_energy_list = []

batch_size = 5
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device= 'cpu'
logging.basicConfig(level=logging.INFO)
model = get_model(0, logging, device)
# modelFile = "../saved_models/re_dock.pt"
# self-dock model
modelFile = "../saved_models/self_dock.pt"

model.load_state_dict(torch.load(modelFile, map_location=device))
_ = model.eval()

data_loader = DataLoader(dataset, batch_size=batch_size, follow_batch=['x', 'y', 'compound_pair'], shuffle=False, num_workers=8)
affinity_pred_list = []
y_pred_list = []
for data in tqdm(data_loader):
    data = data.to(device)
    if not use_related:
        y_pred, affinity_pred = model(data)
        affinity_pred_list.append(affinity_pred.detach().cpu())
    else:
        y_pred, affinity_pred, SX_0, SX_1 = model(data)
        affinity_pred_list.append(affinity_pred.detach().cpu())
        bias_list.append(SX_0.detach().cpu())
        batched_energy_list.append(SX_1.detach().cpu())
    if False:
        # we don't need to save the predicted distance map in HTVS setting.
        for i in range(data.y_batch.max() + 1):
            y_pred_list.append((y_pred[data['y_batch'] == i]).detach().cpu())
affinity_pred_list = torch.cat(affinity_pred_list)

09:46:05   5 stack, readout2, pred dis map add self attention and GVP embed, compound model GIN
Parameter containing:
tensor([1.], requires_grad=True)


100%|██████████| 423/423 [00:25<00:00, 16.48it/s]


In [98]:
info = dataset.data
info['affinity'] = affinity_pred_list

In [99]:
info.to_csv(f"{pre}/TBAff_{target_name}_{pdb_name}_All.csv")

In [100]:
if given_pocket:
    info_given_pocket = info[info["pocket_name"]==given_pocket]
    info_given_pocket.to_csv(f"{pre}/TBAff_{target_name}_{pdb_name}_Given_{given_pocket}.csv")

In [101]:
chosen = info.loc[info.groupby(['protein_name', 'smiles'],sort=False)['affinity'].agg('idxmax')].reset_index()
chosen.to_csv(f"{pre}/TBAff_{target_name}_{pdb_name}_TBChosen.csv")

In [102]:
info.to_csv(f"{pre}/result_info.csv")
if given_pocket:
    info_right_pocket = info[info["pocket_name"]==right_pocket]
    info_right_pocket.to_csv(f"{pre}/result_info_rightpocket.csv")

## Related-Atoms Process

In [103]:
use_related = False
if use_related:
    energy_list = []
    for _batch in batched_energy_list:
        for item in _batch:
            energy_list.append(item)
            
    related_atoms_list = []
    num_atoms_list = []
    for p in info.iterrows():
        sm = p[1]["smiles"]
        #print(sm)
        smiles = get_canonical_smiles(sm)
        #print(smiles)
        mol = Chem.MolFromSmiles(smiles)

        related = []
        for _atom in mol.GetAtoms():
            if _atom.GetAtomicNum() in [7, 8]:
                related.append(_atom.GetIdx())
        related_atoms_list.append(related)
        num_atoms_list.append(mol.GetNumHeavyAtoms())
    
    colsum_energy_list = [i.sum(axis=0) for i in energy_list]
    related_energy_list = [colsum_energy_list[i][related_atoms_list[i]].sum() for i in range(len(colsum_energy_list))]
    total_energy_list = [i.sum() for i in energy_list]

    related_energy_tensor = torch.tensor(related_energy_list)
    total_energy_tensor = torch.tensor(total_energy_list)
    nonrelated_energy_tensor = total_energy_tensor - related_energy_tensor

    bias = bias_list[0]
    outleaky = torch.nn.LeakyReLU()

    related_affinity = outleaky(bias + related_energy_tensor)
    nonrelated_affinity = outleaky(bias + nonrelated_energy_tensor)
    
    info['related_affinity'] = related_affinity
    info['nonrelated_affinity'] = nonrelated_affinity
    
    related_chosen = info.loc[info.groupby(['protein_name', 'smiles'],sort=False)['related_affinity'].agg('idxmax')].reset_index()
    nonrelated_chosen = info.loc[info.groupby(['protein_name', 'smiles'],sort=False)['nonrelated_affinity'].agg('idxmax')].reset_index()
    related_chosen.to_csv(f"{pre}/result_chosen_related.csv")

# 以下暂时不用

In [104]:
from generation_utils import get_LAS_distance_constraint_mask, get_info_pred_distance, write_with_new_coords
# pick one with affinity greater than 7.
chosen = info.loc[info.groupby(['protein_name', 'smiles'],sort=False)['affinity'].agg('idxmax')].reset_index()
chosen = chosen.query("affinity > 7").reset_index(drop=True)
line = chosen.iloc[0]
idx = line['index']
one_data = dataset[idx]
data_with_batch_info = next(iter(DataLoader(dataset[idx:idx+1], batch_size=1, 
                         follow_batch=['x', 'y', 'compound_pair'], shuffle=False, num_workers=1)))
y_pred, affinity_pred = model(data_with_batch_info)

coords = one_data.coords.to(device)
protein_nodes_xyz = one_data.node_xyz.to(device)
n_compound = coords.shape[0]
n_protein = protein_nodes_xyz.shape[0]
y_pred = y_pred.reshape(n_protein, n_compound).to(device).detach()
y = one_data.dis_map.reshape(n_protein, n_compound).to(device)
compound_pair_dis_constraint = torch.cdist(coords, coords)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper__index_select)

In [ ]:
smiles = line['smiles']
print(smiles)
mol = Chem.MolFromSmiles(smiles)
mol.Compute2DCoords()
LAS_distance_constraint_mask = get_LAS_distance_constraint_mask(mol).bool()
info = get_info_pred_distance(coords, y_pred, protein_nodes_xyz, compound_pair_dis_constraint, 
                              LAS_distance_constraint_mask=LAS_distance_constraint_mask,
                              n_repeat=1, show_progress=False)
toFile = f'{base_pre}/one_tankbind.sdf'
new_coords = info.sort_values("loss")['coords'].iloc[0].astype(np.double)
write_with_new_coords(mol, new_coords, toFile)

In [ ]:
import nglview   # conda install nglview -c conda-forge if import failure

proteinName = "6dlo"
proteinFile = f"{base_pre}/{proteinName}.pdb"
view = nglview.show_file(nglview.FileStructure(proteinFile), default=False)
view.add_representation('cartoon', selection='protein', color='white')

predictedFile = f'{base_pre}/one_tankbind.sdf'
rdkit = view.add_component(nglview.FileStructure(predictedFile), default=False)
rdkit.add_ball_and_stick(color='red')
view

In [ ]:
view.render_image()

In [ ]:
view._display_image()